In [41]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
from time import sleep
from time import time
from random import randint
from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
import re


# URL Structure

In [2]:
url="https://www.imdb.com/search/title/?year=2000-01-01,2000-12-31&sort=num_votes,desc&ref_=adv_prv"
response=r.get(url)
soup=BeautifulSoup(response.text,"html.parser")

In [3]:
response=r.get(url)

In [4]:
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


## HTML page structre

In [5]:
soup=BeautifulSoup(response.text,"html.parser")

In [6]:
movies_content=soup.find_all("div",class_="lister-item mode-advanced")
print(type(movies_content))
print(len(movies_content))
#verifier qu'on a bien les 50 films dans la page1

<class 'bs4.element.ResultSet'>
50


# extra data of 1 movie

In [7]:
first_movie=movies_content[0]


In [8]:
#titre du film
#i used dev tools of chrome to help me select my item
first_name=first_movie.h3.a.text
first_name

'Gladiator'

In [9]:
first_year=first_movie.h3.find('span',class_="lister-item-year text-muted unbold")
print(first_year.text)


(2000)


In [10]:
first_movie.strong

<strong>8.5</strong>

In [11]:
#note IMBD
#convertir en float
first_IMBD_note = float(first_movie.strong.text)
print(first_IMBD_note)


8.5


In [12]:
first_metascore=first_movie.find('span', class_="metascore favorable")
first_metascore= int(first_metascore.text)
print(first_metascore)

67


In [13]:
#nombre de votes

first_votes= first_movie.find('span', attrs={'name':'nv'})
first_votes


<span data-value="1244723" name="nv">1,244,723</span>

In [14]:
first_votes['data-value']


'1244723'

In [15]:
first_votes=int(first_votes['data-value'])
first_votes

1244723

# scrape one page

In [16]:
names=[]
years=[]
imbd_ratings=[]
metascores=[]
votes=[]
#on reprend notre data_movie content pour y extraire l'information   #if the movie has a metascore =>extract it
for container in movies_content:
    name=container.h3.a.text
    names.append(name)   #on ajoute chaque element name de la boucle a la liste names
    year=container.h3.find('span',class_="lister-item-year").text
    years.append(year)
    imbd=float(container.strong.text)
    imbd_ratings.append(imbd)
    m_score=container.find('span', class_="metascore favorable")
    #m_score= int(m_score).text
    metascores.append(m_score)  
    vote=container.find('span', attrs={'name':'nv'})['data-value']
    votes.append(int(vote))
    
print(type(m_score))
       

<class 'bs4.element.Tag'>


# affichage en dataframe avec pandas

In [18]:
test_df=pd.DataFrame({'Movie':names, 'year':years, 'ratings':imbd_ratings, 'votes':votes, 'metascore':metascores})
print(test_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
Movie        0 non-null float64
year         0 non-null float64
ratings      0 non-null float64
votes        0 non-null float64
metascore    0 non-null float64
dtypes: float64(5)
memory usage: 76.0 bytes
None


# scrape all pages

In [3]:
pages=[str(i) for i in range(1,5)]
pages

['1', '2', '3', '4']

In [4]:
years_url=[str(i) for i in range(1994,2006)]
years_url

['1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005']

### controlling requests_asked number

In [5]:
for a in range(1,5):
    print('wakeup')
    sleep(randint(1,4))
    

wakeup
wakeup
wakeup
wakeup


In [6]:
start_time = time()  #on fixe le temps de début
requests=0           #count requests
for _ in range(5):
    requests+=1
    sleep(randint(1,3))
    elapsed_time=time()- start_time #count time passed since the 1st request
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait=True)

Request: 5; Frequency: 0.4156129369879545 requests/s


In [7]:
from warnings import warn
warn('Attention')

C:\Users\33622\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Attention
  


# Final script


In [19]:
#redeclarer les listes
names = []
years= []
ratings = []
metascores = []
votes = []
imbd_ratings = []
2#preparer l'affichage des boucles
start_time= time()
requests= 0
3#for every year between 1994-2005 i make a loop for every page between 1 and 4 + request get
for year_url in years_url:
    for page in pages:
        response= r.get('https://www.imdb.com/search/title/?release_date='+ year_url +'&sort=num_votes,asc&page='+page)
        4#pause sur la boucle de 8 a 15 sec
        sleep(randint(3,6))
        4#afficher les info sur les requests
        requests += 1
        elapsed_time=time()- start_time
        print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait=True)
        4# waring if code_status is #200
        if response.status_code !=200:
            warn('Request:{}; status code:{}'.format(requests,response.status_code))
            5#stopper la boucle si le nombre de requests >72
        if requests>72:
            warn('many requests,slow down')
            break
        6# extract HTML with Bs4
        page_html= BeautifulSoup(response.text, 'html.parser')
        7#select 50 movies of every page
        mv_containers=page_html.find_all("div",class_="lister-item mode-advanced")
        8#loop for every container
        for container in mv_containers:
            name=container.h3.a.text
            names.append(name)   #on ajoute chaque element name de la boucle a la liste names
            year=container.h3.find('span',class_="lister-item-year").text
            years.append(year)
#             imbd=[float(i.text) for i in page_html.select('span.rating-star.imdb-rating+strong')]
#             imbd_ratings.append(imbd)
            m_score=container.find('span', class_="metascore")
            if m_score is None:
                metascores.append(0)
            else:
                metascores.append(int(m_score.text))
            vote=container.find('span', attrs={'name':'nv'})['data-value']
            votes.append(int(vote))


KeyboardInterrupt: 

In [88]:
#redeclarer les listes
names = []
years= []
ratings = []
metascores = []
votes = []
imbd_ratings = []
2#preparer l'affichage des boucles
start_time= time()
requests= 0
3#for every year between 1994-2005 i make a loop for every page between 1 and 4 + request get
for year_url in years_url:
    for page in pages:
        response= r.get('https://www.imdb.com/search/title/?release_date='+ year_url +'&sort=num_votes,asc&page='+page)
        4#pause sur la boucle de 8 a 15 sec
        sleep(randint(3,6))
        4#afficher les info sur les requests
        requests += 1
        elapsed_time=time()- start_time
        print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait=True)
        4# waring if code_status is #200
        if response.status_code !=200:
            warn('Request:{}; status code:{}'.format(requests,response.status_code))
            5#stopper la boucle si le nombre de requests >72
        if requests>72:
            warn('many requests,slow down')
            break
        6# extract HTML with Bs4
        page_html= BeautifulSoup(response.text, 'html.parser')
        imbd=[float(i.text) for i in page_html.select('div.inline-block.ratings-imdb-rating')]
        imbd1 = [j for j in imbd]
        imbd_ratings.append(imbd1)
        7#select 50 movies of every page
        mv_containers=page_html.find_all("div",class_="lister-item mode-advanced")
        8#loop for every container
        for container in movies_content:
            name=container.h3.a.text
            names.append(name)   #on ajoute chaque element name de la boucle a la liste names
            year=container.h3.find('span',class_="lister-item-year").text
            years.append(year)
#             imbd=[float(i.text) for i in page_html.select('div.inline-block.ratings-imdb-rating')]
#             print(imbd)
#             imbd_ratings.append(imbd)
#             print(imbd_ratings)
            m_score=container.find('span', class_="metascore")
            if m_score is None:
                metascores.append(0)
            else:
                metascores.append(int(m_score.text))
            vote=container.find('span', attrs={'name':'nv'})['data-value']
            votes.append(int(vote))


Request: 48; Frequency: 0.15258240982787954 requests/s


In [39]:
#to fetch imdb ratings
imbd_ratings = []
for year_url in years_url:
    for page in pages:
        response= r.get('https://www.imdb.com/search/title/?release_date='+ year_url +'&sort=num_votes,asc&page='+page)
        4#pause sur la boucle de 8 a 15 sec
        sleep(randint(3,6))
        4#afficher les info sur les requests
        requests += 1
        elapsed_time=time()- start_time
        print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait=True)
        4# waring if code_status is #200
        if response.status_code !=200:
            warn('Request:{}; status code:{}'.format(requests,response.status_code))
            5#stopper la boucle si le nombre de requests >72
        if requests>72:
            warn('many requests,slow down')
            break
        6# extract HTML with Bs4
        page_html= BeautifulSoup(response.text, 'html.parser')
        imbd=[float(i.text) for i in page_html.select('div.inline-block.ratings-imdb-rating')]
        imbd1 = [j for j in imbd]
        imbd_ratings.append(imbd1)


SSLError: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /search/title/?release_date=1994&sort=num_votes,asc&page=1 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')")))

In [40]:
imbd_ratings

[]

In [26]:
 7#select 50 movies of every page
for year_url in years_url:
    for page in pages:
        mv_containers=page_html.find_all("div",class_="lister-item mode-advanced")
        8#loop for every container
for container in mv_containers:
    name=container.h3.a.text
    names.append(name)   #on ajoute chaque element name de la boucle a la liste names
    year=container.h3.find('span',class_="lister-item-year").text
    years.append(year)
#imbd=[float(i.text) for i in page_html.select('div.inline-block.ratings-imdb-rating')]
#   print(imbd)
#             imbd_ratings.append(imbd)
#             print(imbd_ratings)
    m_score=container.find('span', class_="metascore")
    if m_score is None:
        metascores.append(0)
    else:
        metascores.append(int(m_score.text))
    vote=container.find('span', attrs={'name':'nv'})['data-value']
    votes.append(int(vote))


In [29]:
k = []
for i in range(len(imbd_ratings)):
    for j in imbd_ratings[i]:
        k.append(j)
len(k)

1800

In [38]:
# Movies_rating=pd.DataFrame({'Movie':names, 
#                             'year':years, 
#                             'imbd_ratings':imbmetascoresd_ratings, 
#                             'votes':votes, 
#                             'metascore':metascores})
# #print(Movies_rating.info())
# Movies_rating.head(100)

Movies_rating=pd.DataFrame({'Movie':names, 
                            'year':years, 
                            'votes':votes, 
                            'metascore':metascores
                            'imdb_rating':k})
#print(Movies_rating.info())
Movies_rating.head(20)

SyntaxError: invalid syntax (<ipython-input-38-879785b969d9>, line 13)

In [35]:
Movies_rating.shape

(2400, 5)

In [117]:
imbd_ratings[0]==imbd_ratings[10]

True

In [55]:
'https://www.imdb.com/search/title/?release_date='+ year_url +'&sort=num_votes,asc&page='+page

'https://www.imdb.com/search/title/?release_date=2005&sort=num_votes,asc&page=4'

In [57]:
imbd_ratings[-10:]

[5.9, 6.6, 8.0, 8.1, 7.2, 6.7, 7.2, 5.7, 6.8, 7.3]

for year_url in years_url:
    page_html= BeautifulSoup(response.text, 'html.parser')
    mv_containers=page_html.find_all("div",class_="lister-item.mode-advanced")
    year=container.h3.find('span',class_="lister-item-year").text
    years.append(year)

print(years)

## PIpelines

In [ ]:
title='Top 10 Movies by IMBD ratings between 1994 and 2005'


In [ ]:
result=DataFrame.groupby('Movie',as_index=False)['metascore'].mean().sort_values(by='metascore',ascending=False).head(10).round(1)

In [ ]:
fig,ax=plt.subplots(figsize=(20,10))
sns.barplot(data=result,x='Movie',y='IMDb_ratings')
plt.title(title+'\n',fontsize=16)

sns.set_style('dark')
plt.show()

# Rough

In [67]:
url="https://www.imdb.com/search/title/?year=2000-01-01,2000-12-31&sort=num_votes,desc&ref_=adv_prv"
response=r.get(url)
soup=BeautifulSoup(response.text,"html.parser")
imbd=[float(i.text) for i in soup.select('div.inline-block.ratings-imdb-rating')]
imdb = [j for j in imbd]
imdb


[8.5,
 8.4,
 8.3,
 8.3,
 7.4,
 7.8,
 7.6,
 7.3,
 7.0,
 6.1,
 7.7,
 6.5,
 7.8,
 7.2,
 7.9,
 6.2,
 7.1,
 6.7,
 6.6,
 6.7,
 8.1,
 7.6,
 7.8,
 6.1,
 6.4,
 6.2,
 7.2,
 7.0,
 5.5,
 7.3,
 7.3,
 7.6,
 7.5,
 6.4,
 6.4,
 5.5,
 7.7,
 5.5,
 5.8,
 6.5,
 5.9,
 6.6,
 8.0,
 8.1,
 7.2,
 6.7,
 7.2,
 5.7,
 6.8,
 7.3]